In [1]:
## This Cell is related to date preprocessing and is completely copied from original kipf's github code!

import numpy as np
import pandas as pd
import scipy.sparse as sp
import torch,torch.nn,torch.sparse,torch.nn.functional,torch.distributions
from sklearn.metrics import roc_auc_score,average_precision_score
np.set_printoptions(threshold=np.inf)


from google.colab import drive
drive.mount('/content/drive')


W_gene      =  pd.read_table( "/content/drive/My Drive/Data Fusion/GBM_Adjacency_matrices/gene.txt", delim_whitespace=True).to_numpy()
W_methy     =  pd.read_table( "/content/drive/My Drive/Data Fusion/GBM_Adjacency_matrices/methy.txt", delim_whitespace=True).to_numpy()
W_mirna     =  pd.read_table( "/content/drive/My Drive/Data Fusion/GBM_Adjacency_matrices/mirna.txt", delim_whitespace=True).to_numpy()

gene      =  pd.read_table( "/content/drive/My Drive/Data Fusion/GBM/GLIO_Gene_Expression.txt", delim_whitespace=True).to_numpy().transpose()
methy     =  pd.read_table( "/content/drive/My Drive/Data Fusion/GBM/GLIO_Methy_Expression.txt", delim_whitespace=True).to_numpy().transpose()
mirna     =  pd.read_table( "/content/drive/My Drive/Data Fusion/GBM/GLIO_Mirna_Expression.txt", delim_whitespace=True).to_numpy().transpose()
truelabel =  pd.read_table( "/content/drive/My Drive/Data Fusion/GBM/GLIO_Survival.txt", delim_whitespace=True).to_numpy()



print(gene.shape)
print(methy.shape)
print(mirna.shape)

def get_nomralized_laplacian(a):
    b = np.diag(np.power(1.0 / a.sum(axis=1), 0.5))
    return (b @ a @ b)





features = np.c_[mirna,methy,gene]

patient_no = features.shape[0]

idx = np.arange(patient_no)
np.random.shuffle(idx)
train_idx = idx[0:int(patient_no * 0.75)]
val_idx =   idx[int(patient_no * 0.75):]

train_features = features[train_idx]
val_features   = features[val_idx]
print(train_features.shape)
print(val_features.shape)




Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:14: FutureWarning: read_table is deprecated, use read_csv instead, passing sep='\t'.
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:15: FutureWarning: read_table is deprecated, use read_csv instead, passing sep='\t'.
  from ipykernel import kernelapp as app
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:16: FutureWarning: read_table is deprecated, use read_csv instead, passing sep='\t'.
  app.launch_new_instance()
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:18: FutureWarning: read_table is deprecated, use read_csv instead, passing sep='\t'.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:19: FutureWarning: read_table is deprecated, use read_csv instead, passing sep='\t'.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:20: FutureWarning: read_table is deprecated, use read_csv instead, passing sep='\t'.
/usr/local/lib/python3.6/dist-packages/ipykernel_lau

(215, 12042)
(215, 1305)
(215, 534)
(161, 13881)
(54, 13881)


In [0]:
#Model definition Cell

num_features = features.shape[1]

train_features_tensor  = torch.Tensor(train_features)
val_features_tensor    = torch.Tensor(val_features)





class Model(torch.nn.Module):

    def __init__(self,**kwargs):
        super(Model, self).__init__(**kwargs)
        
        
        
            

        self.W0 = torch.nn.Linear(num_features,6000)
        self.W1 = torch.nn.Linear(6000,3000)
        self.W2 = torch.nn.Linear(3000,6000)
        self.W3 = torch.nn.Linear(6000,num_features)

        
        param_list = []
        param_list += list(self.W0.parameters())
        param_list += list(self.W1.parameters())
        param_list += list(self.W2.parameters())
        param_list += list(self.W3.parameters())
        
        

        self.optimizer = torch.optim.Adam(param_list,lr=0.0001)
        self.recon = None
        
        
        

    


        
    def forward(self,x) : 
        self.layer_1 = torch.nn.functional.tanh(self.W0(x))
        self.layer_2 = torch.nn.functional.tanh(self.W1(self.layer_1))
        self.layer_3 = torch.nn.functional.tanh(self.W2(self.layer_2))
        self.layer_4 = self.W3(self.layer_3)
        


        self.x_prim = self.layer_4
        self.embedding = self.layer_2
        self.recon_loss = ((self.x_prim-x)**2).mean(0)
        self.recon_loss = self.recon_loss[0:534].mean()+self.recon_loss[534:534+1305].mean()+self.recon_loss[534+1305:534+1305+12042].mean()
        self.loss = self.recon_loss/3
        
        
        
        
        return self.loss.item()
        
    def backward(self) :
        
        self.optimizer.zero_grad()
        self.loss.backward()
        self.optimizer.step()

        
        
    def get_cosine_sim_emb(self):
        
        emb = self.embedding.detach().numpy()

        norms =  (emb*emb).sum(1).reshape(-1,1)**0.5
        normalized_emb = emb/norms

        emb_matrix = np.abs( normalized_emb@normalized_emb.transpose())
        return emb_matrix
        

    def get_euclidian(self):
        emb = self.embedding.detach().numpy()
        
        emb_matrix = np.zeros((emb.shape[0],emb.shape[0]))
        
        for i in range(emb.shape[0]):
            for j in range(emb.shape[0]):
                emb_matrix[i,j] = ((emb[i]-emb[j])**2).sum()
                
        return emb_matrix
        
        



In [4]:
#Training Cell 

my_model = Model()


for epoch_counter in range(20000):
    
    train_loss = my_model.forward(train_features_tensor)
    my_model.backward()
    val_loss = my_model.forward(val_features_tensor)
    #val_loss = 0
    
    if epoch_counter % 1 ==0:
        print("Epoch:", '%04d' % (epoch_counter + 1), "train_loss=", "{:.5f}".format(train_loss), "val_loss=", "{:.5f}".format(val_loss))


print('\n',"#########################################################",'\n')






/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:1374: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")


Epoch: 0001 train_loss= 0.98149 val_loss= 1.01913
Epoch: 0002 train_loss= 0.91933 val_loss= 0.96986
Epoch: 0003 train_loss= 0.85777 val_loss= 0.92422
Epoch: 0004 train_loss= 0.79896 val_loss= 0.88532
Epoch: 0005 train_loss= 0.74468 val_loss= 0.84974
Epoch: 0006 train_loss= 0.69468 val_loss= 0.81888
Epoch: 0007 train_loss= 0.65126 val_loss= 0.79389
Epoch: 0008 train_loss= 0.61373 val_loss= 0.77461
Epoch: 0009 train_loss= 0.58159 val_loss= 0.75439
Epoch: 0010 train_loss= 0.55165 val_loss= 0.73738
Epoch: 0011 train_loss= 0.52507 val_loss= 0.72540
Epoch: 0012 train_loss= 0.50143 val_loss= 0.71572
Epoch: 0013 train_loss= 0.47935 val_loss= 0.70587
Epoch: 0014 train_loss= 0.45841 val_loss= 0.69579
Epoch: 0015 train_loss= 0.43939 val_loss= 0.68724
Epoch: 0016 train_loss= 0.42167 val_loss= 0.68101
Epoch: 0017 train_loss= 0.40482 val_loss= 0.67564
Epoch: 0018 train_loss= 0.38864 val_loss= 0.66943
Epoch: 0019 train_loss= 0.37324 val_loss= 0.66531
Epoch: 0020 train_loss= 0.35860 val_loss= 0.66036


KeyboardInterrupt: ignored

In [5]:
#my_model.forward(features_tensor)
emb = my_model.embedding.detach().numpy()
        
emb_matrix = np.zeros((emb.shape[0],emb.shape[0]))
        
for i in range(emb.shape[0]):
    for j in range(emb.shape[0]):
        #emb_matrix[i,j] =  ((emb[i]-emb[j])**2).sum()
        emb_matrix[i,j] =  np.exp(-((emb[i]-emb[j])**2).sum()/120)
                
print(emb_matrix)

[[1.00000000e+00 1.39424487e-13 1.86622969e-10 4.10625404e-12
  5.61895200e-11 2.10006603e-12 3.48861159e-13 1.45774736e-13
  4.22185264e-12 1.75390349e-12 9.75258817e-11 2.13034854e-12
  1.17035328e-11 3.11199207e-12 4.43010780e-13 4.35706295e-10
  1.64816077e-11 3.80834155e-12 7.93917112e-12 2.18589676e-12
  3.85258539e-11 6.09656825e-12 4.11252447e-12 1.76640261e-11
  3.08242900e-11 6.56209568e-12 5.73251164e-12 2.65677663e-11
  1.84064736e-11 9.49631257e-13 1.26422422e-13 6.90144740e-12
  1.54224239e-11 9.26085620e-12 3.00809783e-11 7.70017049e-12
  7.24732934e-14 1.45348102e-12 5.16921476e-13 2.06839947e-12
  2.08832212e-13 2.29991580e-12 8.10120474e-13 4.01068090e-13
  4.56522342e-13 2.42381726e-13 1.48901833e-12 9.94108203e-13
  3.89930125e-13 2.87092301e-12 7.06920829e-12 1.86270811e-11
  2.38697447e-12 4.52877855e-12 4.84016740e-13 2.56355328e-13
  3.05083459e-11 1.16091628e-12 7.27036951e-11 4.10656525e-14
  1.05342254e-11 2.12992979e-11 4.81599366e-13 1.50310965e-13
  1.2328

In [0]:
my_model.forward(train_features_tensor)
emb = my_model.embedding.detach().numpy()
np.savetxt("/content/drive/My Drive/Data Fusion/results/emb_v11.txt",emb,delimiter='\t')
#np.savetxt("/content/drive/My Drive/Data Fusion/gvae_fusion_results/res_sade.txt",emb_matrix,delimiter='\t')

/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:1374: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")


In [0]:
from sklearn.cluster import KMeans
X = emb
n_clusters=3
kmeans = KMeans(n_clusters=n_clusters, random_state=0).fit(X)

for i in range(n_clusters):
    indx = np.argwhere(kmeans.labels_==i).reshape(-1)
    print(truelabel[indx][:,1].mean())



626.8765432098766
536.3030303030303
573.3168316831683


In [0]:
import numpy as np
from google.colab import drive
drive.mount('/content/drive')
emb = np.loadtxt("/content/drive/My Drive/Data Fusion/results/emb_v11.txt",delimiter='\t')


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
emb_matrix = my_model.get_cosine_sim_emb()
print(emb_matrix)
np.savetxt("gvae_fusion_results/res_cosine.txt",emb_matrix,delimiter='\t')

[[9.99999762e-01 1.07528433e-01 2.69530844e-02 9.13520753e-02
  1.03215739e-01 7.80488998e-02 5.88160381e-02 2.92040147e-02
  3.22823524e-02 2.40955632e-02 2.25115180e-01 5.83793297e-02
  6.07349798e-02 7.87358359e-02 9.43368226e-02 6.90505430e-02
  2.34133713e-02 4.30251881e-02 3.36033434e-01 3.34922850e-01
  1.67553216e-01 1.98978633e-02 2.84192488e-02 2.92781666e-02
  1.31397210e-02 3.53594646e-02 2.65772641e-01 4.52555791e-02
  4.13791835e-02 2.36687094e-01 9.38652232e-02 1.55723453e-01
  2.88900912e-01 2.28089005e-01 5.03865629e-03 2.44999051e-01
  1.20611086e-01 2.92277858e-02 1.25792027e-01 4.08527851e-02
  2.19282031e-01 1.06328651e-01 9.04962942e-02 2.38926888e-01
  1.54283881e-01 2.10222602e-01 1.37868032e-01 2.08886683e-01
  2.64776915e-01 2.95316100e-01 6.46592751e-02 5.15146069e-02
  4.34365980e-02 1.80428043e-01 1.37596533e-01 1.18417107e-01
  1.36523679e-01 8.89368914e-03 1.69216335e-01 1.62840560e-02
  3.34962867e-02 1.38020709e-01 1.60055198e-02 5.96024096e-02
  1.8452